In [1]:
# Create Graph From file SNN_metric
import create_snnGraph as cs
import networkx as nx
import sparsify_snnGraph as ss

a = cs.readData('../snn_computation/snn_metric.csv')
# a = cs.readData('../../SNN-density/snn_computation/snn_metric_ds2.csv')
G = cs.createGraph(a)

print("G edges: ",G.number_of_edges())
print("G nodes: ",G.number_of_nodes())


G edges:  7375
G nodes:  9498


In [2]:
# Silhoutte Coefficient
import pandas as pd
import numpy as np
from os import path
import sys
sys.path.append('../../')
from Evaluation.silhouette_coefficient import silhoutte_result
def read_snnArray(file_path=None):
    df = pd.read_csv(file_path,index_col=False,header=None,sep=' ')
    snn_nparray = np.array(df)
    return snn_nparray
snn_nparray = read_snnArray('../snn_computation/snn_distance_metric.csv')
# snn_nparray2 = read_snnArray('../../SNN-density/snn_computation/snn_distance_metric_ds2.csv')

In [3]:
def demo(G:nx.Graph,distance_metrix,threshold,n):
    G1 =  G.copy()
    snn_nparray = distance_metrix
    label_dictionary = {}
    for j in range (G.number_of_nodes()):
        label_dictionary[j]=-1
    # Sparsify Graph
    g = ss.sparsifyGraph(G1,threshold=threshold)
    sub_graphs = nx.connected_components(g)

    # Create labels for all nodes ,  label -1 for outlier
    i = 0
    for sub in sub_graphs:
        if(len(sub)>n):
            # print(i)
            # print(sub)
            for node in sub:
                label_dictionary[node]=i 
            i +=1
    labels = list(label_dictionary.values())

# Remove outliers

    l = np.array(labels)
    count = 0
    for item in l:
        if item>-1:
            count = count+1
    new_distance_metric = np.zeros((count,count),dtype=float)
    new_distance_metric.shape
    dictionary_no_outliers = {i:label_dictionary[i] for i in label_dictionary if label_dictionary[i]!=-1}

    k = dictionary_no_outliers.keys()
    
    # Create a new name for each node after remove outliers
    change_name_list={}
    start = 0
    for item in k:
        change_name_list[start] = item
        start = start+1

    #Add value to new snn_array, to use for calculate  Silhoutte - after remove outliers
    i=0
    j=0
    for i in range(0,count):
        for j in range(i+1,count):
            new_distance_metric[i][j] = snn_nparray[change_name_list[i]][change_name_list[j]]
            new_distance_metric[j][i] = new_distance_metric[i][j]
     # Silhoutte 
    new_label = list(dictionary_no_outliers.values())
    number_of_cluster = len(set(new_label)) 
    if len(set(new_label)) > 1:
        silhoute_removeouliers= silhoutte_result(new_distance_metric,new_label)
    else:
        print("Number of labels needs to > 1. ERROR!!")
        silhoute_removeouliers = -1

    return silhoute_removeouliers,len(new_label),number_of_cluster,label_dictionary


In [4]:
threshold_list = [i for i in range (1,30)]
for item in threshold_list:
    x1,x2,x3,x4 = demo(G,snn_nparray,item,1)
    print('Threshold: ',item,'score:',x1,'points:', x2,'clusters:', x3)

Threshold:  1 score: 0.022828595481381782 points: 4704 clusters: 509
Threshold:  2 score: 0.09639488660281718 points: 2014 clusters: 576
Threshold:  3 score: 0.20145674999047672 points: 838 clusters: 272
Threshold:  4 score: 0.34814728315897353 points: 426 clusters: 145
Threshold:  5 score: 0.46860608827862343 points: 293 clusters: 96
Threshold:  6 score: 0.5216897677942888 points: 248 clusters: 78
Threshold:  7 score: 0.5437769251459266 points: 227 clusters: 68
Threshold:  8 score: 0.5742705145674987 points: 210 clusters: 61
Threshold:  9 score: 0.5918509261529006 points: 196 clusters: 55
Threshold:  10 score: 0.6394588041227534 points: 181 clusters: 52
Threshold:  11 score: 0.6568614967365756 points: 167 clusters: 46
Threshold:  12 score: 0.6664600668535048 points: 157 clusters: 42
Threshold:  13 score: 0.691094855222317 points: 144 clusters: 38
Threshold:  14 score: 0.7189566755760081 points: 135 clusters: 36
Threshold:  15 score: 0.7550900773252771 points: 120 clusters: 32
Threshol

In [5]:
x1,x2,x3,x4 = demo(G,snn_nparray,19,1)
from collections import defaultdict
v = defaultdict(list)

for key, value in sorted(x4.items()):
    v[value].append(key)
for key in v.keys():
    if (key!=-1):
        print("Cluster: ",key)
        nodes = v[key]
        print(nodes)

Cluster:  0
[1257, 1266, 1319, 7770, 7777, 7796, 7802, 7803, 7812, 7827, 7830, 7833, 7834, 7847, 7856, 7858, 7872, 7875]
Cluster:  1
[1430, 7644]
Cluster:  2
[3341, 3407, 3454]
Cluster:  3
[3679, 3740]
Cluster:  4
[3811, 4826]
Cluster:  5
[3852, 4783]
Cluster:  6
[4407, 9497]
Cluster:  7
[5557, 5562]
Cluster:  8
[5792, 5816]
Cluster:  9
[6007, 6027, 6099, 6129, 8261, 8295]
Cluster:  10
[6110, 6211, 6292]
Cluster:  11
[6497, 6504, 6529]
Cluster:  12
[6582, 6594]
Cluster:  13
[6698, 6703, 6712, 6750, 6774, 6776, 8990]
Cluster:  14
[6823, 8059]
Cluster:  15
[7748, 7757]
Cluster:  16
[8340, 8357]
Cluster:  17
[8400, 8432, 8435, 8462, 8558]
Cluster:  18
[8864, 8878, 8905]
Cluster:  19
[9201, 9218]


In [29]:
# Function to count How many nodes that every items in a list shared as neighbors
from functools import reduce

def count_sharing_nodes(input_list:list,path_knn=''):
    count = 0
    size = len(input_list)
    
    list_of_list = list()
    df = pd.read_csv(path_knn,header=None,sep=' ')
    # df = df.drop(['Unnamed: 0'], axis=1)
    # print(df)
    for node in input_list:
        temp_list = df.loc[node].values.tolist()
        list_of_list.append(temp_list)
    # print(list_of_list)
    res = list(set.intersection(*map(set, list_of_list)))
    return res
# a=[1257, 1266, 1319, 7770, 7777, 7796, 7802, 7803, 7812, 7827, 7830, 7833, 7834, 7847, 7856, 7858, 7872, 7875]
b = [1257, 1266]
res = count_sharing_nodes(b,'../knn_metrics/knn_metric.csv')
print("number of shared neighbors between ",b[0],b[1],' is:',len(res))
print(res)

c = [1257, 7872]
res = count_sharing_nodes(c,'../knn_metrics/knn_metric.csv')
print("number of shared neighbors between ",c[0],c[1],' is:',len(res))
print(res)

number of shared neighbors between  1257 1266  is: 19
[7812, 7827, 1430, 7830, 9497, 7833, 7834, 1319, 7847, 7856, 7858, 7872, 7875, 7748, 7770, 7777, 7796, 7802, 7803]
number of shared neighbors between  1257 7872  is: 19
[7812, 7827, 1430, 7830, 9497, 7833, 7834, 1319, 7847, 7856, 7858, 7875, 7748, 7770, 7777, 1266, 7796, 7802, 7803]
